# Project Resonance: A-Z Benchmark Notebook
**Version:** 1.0
**Date:** June 11, 2025

## Objective
This notebook contains the complete workflow to test our core hypothesis: that a novel, resonance-based reward signal can fine-tune the Evo-1B foundation model to outperform its baseline on the BRCA1 variant effect prediction task. Success is defined as our tuned model achieving a higher Area Under the Curve (AUC) score than the baseline.

## Part 1: Initial Environment Setup

This notebook assumes you have already performed the following one-time setup steps in your terminal on the cloud instance:

1. **Initialized Conda:**
   ```bash
   ~/miniconda3/bin/conda init bash
   source ~/.bashrc
   ```
2. **Created and Activated the Environment:**
   ```bash
   conda create -n evo_project python=3.10 -y
   conda activate evo_project
   ```
3. **Installed Libraries:** You have run the `pip install` commands for PyTorch, Transformers, PEFT, etc.
4. **Logged into Hugging Face:** You have run `huggingface-cli login` and provided your access token.

---

## Part 2: Imports and Global Configuration

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tqdm.notebook import tqdm
import os

# --- Global Configuration ---
MODEL_ID = "togethercomputer/evo-1-131k-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Fine-tuning Hyperparameters
LEARNING_RATE = 1e-4
BATCH_SIZE = 8
NUM_EPOCHS = 1
LAMBDA_DIVERSITY = 0.1 # Weight for our diversity reward term
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.1

# File Paths (You may need to create these files)
STABILITY_DATASET_PATH = "chr22.fa" # Dataset for our fine-tuning
BRCA1_REF_PATH = "brca1_reference.fa" # Reference sequence for BRCA1
BRCA1_VARIANTS_PATH = "brca1_variants.csv" # ClinVar data for BRCA1
TUNED_LORA_PATH = "./resonator_lora"
TUNED_HEAD_PATH = "./resonator_head.pth"

print(f"Using device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Part 3: Data Preparation & Model Architecture

This section contains helper functions and classes for loading data and defining our custom model components. We will call these later.

In [ ]:
class FastaDataset(Dataset):
    """A simple PyTorch Dataset for FASTA files."""
    def __init__(self, fasta_file, tokenizer, max_length=1024):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.sequences = []
        if not os.path.exists(fasta_file):
            print(f"Warning: Fasta file not found at {fasta_file}. Creating dummy file.")
            with open(fasta_file, 'w') as f:
                f.write(">dummy_sequence\n")
                f.write("GATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACA\n")
        
        with open(fasta_file, 'r') as f:
            sequence = ""
            for line in f:
                if line.startswith('>'):
                    if sequence: self.sequences.append(sequence)
                    sequence = ""
                else:
                    sequence += line.strip().upper()
            if sequence: self.sequences.append(sequence)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        tokenized = self.tokenizer(seq, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        return {key: val.squeeze(0) for key, val in tokenized.items()} # Remove batch dimension

class ProjectionHead(nn.Module):
    """Our custom projection head to create a clean latent space."""
    def __init__(self, input_dim=4096, hidden_dim=512, output_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

def calculate_resonance_reward(latent_vectors):
    """Calculates the reward for a batch of latent vectors."""
    # Reward for Low Entropy (approximated by rewarding vectors pushed from origin)
    reward_entropy = torch.linalg.norm(latent_vectors, dim=1).mean()

    # Reward for Diversity (high variance across the batch)
    reward_diversity = latent_vectors.var(dim=0).mean()
    
    total_reward = reward_entropy + LAMBDA_DIVERSITY * reward_diversity
    
    # The loss for the optimizer is the negative of the reward we want to maximize
    loss = -total_reward
    
    return loss, reward_entropy.item(), reward_diversity.item()

print("Helper classes and functions defined.")

---

## Part 4: The Benchmark Execution

This section contains the main workflow for our experiment. We will execute these cells sequentially to get our final result.

### Step 4.1: Data Acquisition & Preparation
**Action:** Create the necessary data files. For this first run, we will create dummy files. You should replace these with real, curated data for the final experiment.

In [ ]:
# Create a dummy stability dataset for fine-tuning
with open(STABILITY_DATASET_PATH, 'w') as f:
    f.write(">highly_conserved_region_1\n")
    f.write("AGCTCGGGTTAAACTAGCGGTCGATCGGCTAGCTAGCTACGCTAGCTACGCTAGCT\n")
    f.write(">highly_conserved_region_2\n")
    f.write("TATATATACGCGCTATATACGCGCGTATATACGCGCGTATATACGCGCTATACG\n")

# Create a dummy BRCA reference fasta
with open(BRCA1_REF_PATH, 'w') as f:
    f.write(">brca1_ref\n")
    f.write("GATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACATTTTTTATACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACAGATTACA" * 10)
    
# Create a dummy BRCA variants CSV
dummy_variants = {
    'position': [50, 100],
    'ref_allele': ['A', 'T'],
    'alt_allele': ['G', 'C'],
    'label': ['Pathogenic', 'Benign']
}
pd.DataFrame(dummy_variants).to_csv(BRCA1_VARIANTS_PATH, index=False)

print(f"Dummy data files created at:")
print(f"- {STABILITY_DATASET_PATH}")
print(f"- {BRCA1_REF_PATH}")
print(f"- {BRCA1_VARIANTS_PATH}")

### Step 4.2: Fine-Tuning Our Model (The Experiment)
**Action:** Run the custom fine-tuning loop to create our specialized "Stability Detector".

In [ ]:
print("--- Starting Model Fine-Tuning ---")

# 1. Load Tokenizer and Dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
stability_dataset = FastaDataset(STABILITY_DATASET_PATH, tokenizer)
stability_dataloader = DataLoader(stability_dataset, batch_size=BATCH_SIZE)

# 2. Load and Prepare Model
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
for param in base_model.parameters(): param.requires_grad = False

lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=LORA_R, lora_alpha=LORA_ALPHA, lora_dropout=LORA_DROPOUT, target_modules=["Wqkv", "out_proj"])
lora_model = get_peft_model(base_model, lora_config).to(DEVICE)
projection_head = ProjectionHead().to(DEVICE)

# 3. Set up Optimizer
trainable_params = list(lora_model.parameters()) + list(projection_head.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=LEARNING_RATE)

# 4. Run the Training Loop
training_history = []
lora_model.train()
projection_head.train()

for epoch in range(NUM_EPOCHS):
    progress_bar = tqdm(stability_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        
        outputs = lora_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        sequence_embedding = outputs.hidden_states[-1].mean(dim=1)
        latent_vectors = projection_head(sequence_embedding)
        
        loss, _, _ = calculate_resonance_reward(latent_vectors)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        training_history.append(-loss.item())
        progress_bar.set_postfix({"Total Reward": f"{-loss.item():.4f}"})

# 5. Save the tuned components
lora_model.save_pretrained(TUNED_LORA_PATH)
torch.save(projection_head.state_dict(), TUNED_HEAD_PATH)

print("\nFine-tuning complete. Model saved.")

# Plot training reward
plt.figure(figsize=(10, 5))
plt.plot(training_history, label='Total Reward')
plt.title('Resonance-Tuning Reward History')
plt.xlabel('Steps')
plt.ylabel('Reward')
plt.legend()
plt.grid(True)
plt.show()

### Step 4.3: Comparative Analysis (The Final Result)
**Action:** Load both the baseline and our tuned model, score the BRCA benchmark dataset with each, and plot the ROC curves to compare their performance.

In [ ]:
print("--- Starting Benchmark Comparison ---")

# This cell is a placeholder for the full benchmarking script.
# It requires the functions 'get_baseline_scores' and 'get_resonator_scores' to be fully implemented
# as conceptualized in the project plan document.

# For this demonstration, we will generate dummy scores.
print("Generating dummy scores for demonstration purposes...")
num_variants = len(pd.read_csv(BRCA1_VARIANTS_PATH))
y_true = np.random.randint(0, 2, num_variants)

# Dummy baseline scores - should be close to random
y_score_base = y_true * 0.1 + np.random.rand(num_variants) * 0.5

# Dummy resonator scores - should be better than baseline
y_score_res = y_true * 0.4 + np.random.rand(num_variants) * 0.5

# --- ROC Curve Calculation ---
fpr_base, tpr_base, _ = roc_curve(y_true, y_score_base)
auc_base = auc(fpr_base, tpr_base)

fpr_res, tpr_res, _ = roc_curve(y_true, y_score_res)
auc_res = auc(fpr_res, tpr_res)

print(f"Baseline Model AUC: {auc_base:.4f}")
print(f"Our Tuned Model AUC: {auc_res:.4f}")

# --- Plotting ---
plt.figure(figsize=(10, 8))
plt.plot(fpr_base, tpr_base, lw=2, label=f'Baseline Evo-1B (AUC = {auc_base:.4f})', color='blue')
plt.plot(fpr_res, tpr_res, lw=2, label=f'Our Resonance-Tuned Model (AUC = {auc_res:.4f})', color='orange', linestyle='-')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('BRCA1 Variant Pathogenicity Prediction: ROC Curve Comparison')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## Part 5: Conclusion

The final graph above is the primary deliverable of this research sprint. A successful outcome is characterized by the orange line (Our Tuned Model) sitting significantly above the blue line (Baseline Evo-1B), with a correspondingly higher AUC score. This would provide the first piece of strong empirical evidence that our novel fine-tuning paradigm can successfully guide a foundation model to learn and apply a new, underlying principle from data.